In [3]:
import matplotlib.pyplot as plt
import os
import pandas
from skimage import io, transform
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

Step 1 : Load the data

In [4]:
#data_transform = transforms.Compose(
#    [transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))])

data_transform = transforms.Compose(
    [transforms.ToTensor()])

train_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/train_images/', transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_imagenet, batch_size=4, shuffle=True, num_workers=2)

test_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/test_images/', transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_imagenet, batch_size=4, shuffle=True, num_workers=2)

In [5]:
print(train_imagenet.__getitem__(1))

(tensor([[[0.1725, 0.1137, 0.0471,  ..., 0.6235, 0.6353, 0.6039],
         [0.0863, 0.0706, 0.0941,  ..., 0.6471, 0.6392, 0.6078],
         [0.1412, 0.1451, 0.1412,  ..., 0.6000, 0.6000, 0.6157],
         ...,
         [0.2510, 0.1412, 0.0745,  ..., 0.0941, 0.1137, 0.1216],
         [0.1412, 0.1333, 0.1647,  ..., 0.2667, 0.2235, 0.1961],
         [0.4471, 0.5686, 0.6118,  ..., 0.6314, 0.6235, 0.6039]],

        [[0.1725, 0.1137, 0.0471,  ..., 0.6235, 0.6353, 0.6039],
         [0.0863, 0.0706, 0.0941,  ..., 0.6471, 0.6392, 0.6078],
         [0.1412, 0.1451, 0.1412,  ..., 0.6000, 0.6000, 0.6157],
         ...,
         [0.2510, 0.1412, 0.0745,  ..., 0.0941, 0.1137, 0.1216],
         [0.1412, 0.1333, 0.1647,  ..., 0.2667, 0.2235, 0.1961],
         [0.4471, 0.5686, 0.6118,  ..., 0.6314, 0.6235, 0.6039]],

        [[0.1725, 0.1137, 0.0471,  ..., 0.6235, 0.6353, 0.6039],
         [0.0863, 0.0706, 0.0941,  ..., 0.6471, 0.6392, 0.6078],
         [0.1412, 0.1451, 0.1412,  ..., 0.6000, 0.6000, 0

In [6]:
print(len(train_imagenet.__getitem__(1)[0]))

3


In [7]:
print(test_imagenet.__getitem__(7625))

(tensor([[[0.0275, 0.0235, 0.0196,  ..., 0.0392, 0.0275, 0.0353],
         [0.0235, 0.0235, 0.0314,  ..., 0.0549, 0.0275, 0.0314],
         [0.0314, 0.0275, 0.0549,  ..., 0.0745, 0.0314, 0.0235],
         ...,
         [0.9843, 0.9255, 0.9647,  ..., 0.9961, 0.9490, 0.9137],
         [0.9647, 0.9412, 1.0000,  ..., 0.6667, 0.8667, 0.8510],
         [0.9725, 0.9098, 0.9569,  ..., 0.3451, 0.9098, 0.8863]],

        [[0.0275, 0.0235, 0.0196,  ..., 0.0392, 0.0275, 0.0353],
         [0.0235, 0.0235, 0.0314,  ..., 0.0549, 0.0275, 0.0314],
         [0.0314, 0.0275, 0.0549,  ..., 0.0745, 0.0314, 0.0235],
         ...,
         [0.9843, 0.9255, 0.9647,  ..., 0.9961, 0.9490, 0.9137],
         [0.9647, 0.9412, 1.0000,  ..., 0.6667, 0.8667, 0.8510],
         [0.9725, 0.9098, 0.9569,  ..., 0.3451, 0.9098, 0.8863]],

        [[0.0275, 0.0235, 0.0196,  ..., 0.0392, 0.0275, 0.0353],
         [0.0235, 0.0235, 0.0314,  ..., 0.0549, 0.0275, 0.0314],
         [0.0314, 0.0275, 0.0549,  ..., 0.0745, 0.0314, 0

In [8]:
print(train_imagenet)

Dataset ImageFolder
    Number of datapoints: 91720
    Root Location: start_deep/start_deep/start_deep/train_images/
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [9]:
print(train_loader)

In [10]:
print(test_imagenet)

Dataset ImageFolder
    Number of datapoints: 7628
    Root Location: start_deep/start_deep/start_deep/test_images/
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [11]:
print(test_loader)

Step 2 : Define neural network

In [12]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batc
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()

Step 3 : Define a Loss function and optimizer

In [13]:
import torch.optim as optim

weights = torch.Tensor([1,0.5])
criterion = nn.CrossEntropyLoss(weights)
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

Step 4 : Train the network

In [14]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: weight tensor should be defined either for all 10 classes or no classes but got weight tensor of shape: [2] at c:\programdata\miniconda3\conda-bld\pytorch-cpu_1532499824793\work\aten\src\thnn\generic/ClassNLLCriterion.c:29

Step 5 : Test the network

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

classes = (0,1,)
            
for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))